In [19]:
from __future__ import (nested_scopes, generators, division, absolute_import, with_statement,
                        print_function, unicode_literals)


class BTree(object):
    class Node(object):

        def __init__(self, t):
            self.data = []
            self.children = []
            self.leaf = True
            self._t = t

        def split(self, parent, valor):
            new_node = self.__class__(self._t)

            mid_point = self.size//2
            split_value = self.data[mid_point]
            parent.add_data(split_value)

            new_node.children = self.children[mid_point + 1:]
            self.children = self.children[:mid_point + 1]
            new_node.data = self.data[mid_point+1:]
            self.data = self.data[:mid_point]

            if len(new_node.children) > 0:
                new_node.leaf = False

            parent.children = parent.add_child(new_node)
            if valor < split_value:
                return self
            else:
                return new_node

        @property
        def _is_full(self):
            return self.size == 2 * self._t - 1

        @property
        def size(self):
            return len(self.data)

        def add_data(self, valor):
            self.data.append(valor)
            self.data.sort()

        def add_child(self, new_node):
            i = len(self.children) - 1
            while i >= 0 and self.children[i].data[0] > new_node.data[0]:
                i -= 1
            return self.children[:i + 1]+ [new_node] + self.children[i + 1:]
    
    def __init__(self, t):
        self._t = t
        if self._t <= 1:
            raise ValueError("Erro. Precisa ser maior ou igual a 2")
        self.raiz = self.Node(t)

    def insert(self, valor):
        node = self.raiz
        if node._is_full:
            new_raiz = self.Node(self._t)
            new_raiz.children.append(self.raiz)
            new_raiz.leaf = False

            node = node.split(new_raiz, valor)
            self.raiz = new_raiz
        while not node.leaf:
            i = node.size - 1
            while i > 0 and valor < node.data[i] :
                i -= 1
            if valor > node.data[i]:
                i += 1

            next = node.children[i]
            if next._is_full:
                node = next.split(node, valor)
            else:
                node = next

        node.add_data(valor)

    def search(self, valor, node=None):
        if node is None:
            node = self.raiz
        if valor in node.data:
            return True
        elif node.leaf:
            return False
        else:
            i = 0
            while i < node.size and valor > node.data[i]:
                i += 1
            return self.search(valor, node.children[i])

    def print_order(self):
        this_level = [self.raiz]
        while this_level:
            next_level = []
            output = ""
            for node in this_level:
                if node.children:
                    next_level.extend(node.children)
                output += str(node.data) + " "
            print(output)
            this_level = next_level

In [20]:
btree = BTree(3)

In [21]:
btree.insert(1)

In [22]:
btree.insert(3)
btree.insert(6)
btree.insert(8)
btree.insert(14)
btree.insert(32)
btree.insert(36)
btree.insert(39)
btree.insert(41)
btree.insert(43)


In [23]:
btree.print_order()

[6, 32] 
[1, 3] [8, 14] [36, 39, 41, 43] 


In [24]:
btree.search(14)

True

In [25]:
btree.insert(4)
btree.insert(5)
btree.insert(42)
btree.insert(2)
btree.insert(7)

In [26]:
btree.print_order()

[6, 32] 
[1, 2, 3, 4, 5] [7, 8, 14] [36, 39, 41, 42, 43] 
